# Creating "Statistics of field" from the Meteorological Predictor Fields as Input for RF-based ML-Models
Version 18 January 2024, Selina Kiefer

### Input: csv-files
continuous timeseries of meteorological predictors as 2d-fields in csv-format
### Output: csv-file
continuous timeseries of the minimum, mean, maximum and variance of the meteorological predictors per date in csv-format

#### Define the paths' and files' names 

In [ ]:
# Set the needed path and file names.
PATH_defined_functions = './Defined_Functions/'

PATH_input_data = './Data_in_csv_Format/'
ifiles_input_data = ['era5_u10_60W_60E_20N_80N_1950_2020_only_Oct_Apr_lead_time_14d.csv',
                     'era5_z100_60W_60E_20N_80N_1950_2020_only_Oct_Apr_lead_time_14d.csv',
                     'era5_z250_60W_60E_20N_80N_1950_2020_only_Oct_Apr_lead_time_14d.csv',
                     'era5_z500_60W_60E_20N_80N_1950_2020_only_Oct_Apr_lead_time_14d.csv',
                     'era5_z850_60W_60E_20N_80N_1950_2020_only_Oct_Apr_lead_time_14d.csv',
                     'era5_t850_60W_60E_20N_80N_1950_2020_only_Oct_Apr_lead_time_14d.csv',
                     'era5_H850_60W_60E_20N_80N_1950_2020_only_Oct_Apr_lead_time_14d.csv',
                     'era5_u300_60W_60E_20N_80N_1950_2020_only_Oct_Apr_lead_time_14d.csv',
                     'era5_msl_60W_60E_20N_80N_1950_2020_only_Oct_Apr_lead_time_14.csv']

PATH_output_file = './Data_in_csv_Format/'
file_name_output_file = 'era5_statistics_u10_z100_z250_60W_60E_20N_80N_1950_2020_lead_time_14d.csv'

#### Import the necessary packages and functions

In [ ]:
# Import the necessary python packages.
import yaml
import pandas as pd
import numpy as np

In [ ]:
# Import the necessary functions.
import sys
sys.path.insert(1,PATH_defined_functions)
from read_in_csv_data import *

####  List the predictors to be combined

In [ ]:
# List the desired predictors and set how many of these should be taken from the first 
# dataframe. From all other dataframes, only 1 predictor is taken (if more are needed, list
# these input files multiple times in "ifiles_input_data"). It is necessary to take the time as
# a predictor since the data will be grouped by date later.
desired_predictors = ['time', 'month', 'u10', 'z100', 'z250', 'z500', 'z850', 't850', 'H850', 'u300', 'msl']
number_of_predictors_in_first_dataframe = 3
time_column_name = 'time'

#### Combine all predictors into one dataframe

In [ ]:
# A new dataframe is created and the desired predictors from the first data file are written
# into it.
df_combined_input_data = pd.DataFrame()
df_input_data = read_in_csv_data(PATH_input_data, ifiles_input_data[0])
for i in range(number_of_predictors_in_first_dataframe):
    df_combined_input_data[desired_predictors[i]] = df_input_data [desired_predictors[i]]

In [ ]:
# From all other dataframes, the specified predictor is added to this new dataframe.
for k in range(len(ifiles_input_data)-1):
    df_input_data = read_in_csv_data(PATH_input_data, ifiles_input_data[k+1])
    df_combined_input_data[desired_predictors[i+k+1]] = df_input_data [desired_predictors[i+k+1]]

#### Calculate the statistics (minimum, mean, maximum and variance) of the predictor fields

In [ ]:
# Now, the time is set as the index and the data is grouped by date. For every desired statistic
# of the field, the calculation is done directly after the grouping and written as separate
# pandas series. Here, the minimum, mean, maximum and variance are calculated.
df_combined_input_data[time_column_name] = pd.to_datetime(df_combined_input_data[time_column_name])
df_combined_input_data = df_combined_input_data.set_index(time_column_name)
ds_input_data_grouped_min = df_combined_input_data.groupby([df_combined_input_data.index.year, df_combined_input_data.index.month, df_combined_input_data.index.day], as_index=False).min()
ds_input_data_grouped_mean = df_combined_input_data.groupby([df_combined_input_data.index.year, df_combined_input_data.index.month, df_combined_input_data.index.day], as_index=False).mean()
ds_input_data_grouped_max = df_combined_input_data.groupby([df_combined_input_data.index.year, df_combined_input_data.index.month, df_combined_input_data.index.day], as_index=False).max()
ds_input_data_grouped_var = df_combined_input_data.groupby([df_combined_input_data.index.year, df_combined_input_data.index.month, df_combined_input_data.index.day], as_index=False).var()

In [ ]:
# A new dataframe is created combining all statistics and naming them uniquely.
df_statistics = pd.DataFrame()
for l in range(len(desired_predictors)-1):
    df_statistics['min_'+desired_predictors[l+1]] = ds_input_data_grouped_min[desired_predictors[l+1]]
    df_statistics['mean_'+desired_predictors[l+1]] = ds_input_data_grouped_mean[desired_predictors[l+1]]
    df_statistics['max_'+desired_predictors[l+1]] = ds_input_data_grouped_max[desired_predictors[l+1]]   
    df_statistics['var_'+desired_predictors[l+1]] = ds_input_data_grouped_var[desired_predictors[l+1]]   

In [ ]:
# Since the statistics (a single scalar value for each day) of the month are senseless, one of
# the columns is renamed simply with 'month' and the others are removed.
df_statistics = df_statistics.rename(columns={'mean_month':'month'})
df_statistics = df_statistics.drop(['min_month', 'max_month', 'var_month'], axis=1)

#### Add the time information again to the reshaped data

In [ ]:
# Since the time got lost by using .groupby(), a separate new dataframe is created containing
# only the time. To this dataframe, three new columns are added containing the year, month and 
# day.
df_combined_input_data = df_combined_input_data.reset_index()
df_time = pd.DataFrame()
df_time[time_column_name] = pd.to_datetime(df_combined_input_data[time_column_name])
df_time = df_time.set_index(time_column_name)
df_time['year'] = df_time.index.year
df_time['month'] = df_time.index.month
df_time['day'] = df_time.index.day
df_time = df_time.reset_index()

In [ ]:
# This new dataframe is then grouped by date and 'averaged' resulting in a daily time-
# series but separated into year, month and day.
df_time = df_time.set_index(time_column_name)
ds_time_mean = df_time.groupby([df_time.index.year, df_time.index.month, df_time.index.day], as_index=False).mean().astype(int).astype(str) 
df_time_mean = pd.DataFrame(ds_time_mean)

In [ ]:
# The separated timeseries is now combined into a single daily timeseries (nothing needs to be
# changed here).
daily_timeseries = (df_time_mean['year'].astype(str)+'-'+df_time_mean['month'].astype(str)+'-'+df_time_mean['day']).astype(str)

In [ ]:
# In the next step, the time is added to the dataframe containing the statistics as predictors 
# (nothing needs to be changed here).
df_statistics.insert(0, time_column_name, daily_timeseries)

#### Doublecheck the representation of the data

In [ ]:
# Check if everything is reshaped and sorted correctly.
df_statistics.head()

In [ ]:
# Also check if everything is sorted, renamed or removed correctly at the end of the
# dataframe.
df_statistics.tail()

#### Save the data in csv format

In [ ]:
# Save the pandas dataframe in csv-format.
df_statistics.to_csv(PATH_output_file+file_name_output_file)

In [ ]:
# End of Program